<a href="https://colab.research.google.com/github/pulseq/MR-Physics-with-Pulseq/blob/main/tutorials/06_spin_echos/notebooks/Spin_Echo_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Spin Echo Exercise 2**
The sequence has been modified such that the RF pulses are now non-selective and dephasing is achieved using additional gradients. Therefore the minimum time is 2.
Set flip angles such that pure individual signals are generated.
For which signal this fails ?
What would you have to do to achieve clean selection of all individual signals ?




**Initialization**

In [ ]:
try:
  import pypulseq as pp
  print("pypulseq package is readily available\n")

except ImportError or ModuleNotFoundError:
  !pip install git+https://github.com/imr-framework/pypulseq.git
import pypulseq as pp
import math
import warnings
import numpy as np
from matplotlib import pyplot as plt
!pip install MRzeroCore
import MRzeroCore as mr0



# **Create, simulate, and reconstruct sequence**

**Multiecho sequence definition**

In [ ]:
seq_filename = "echoes.seq"


system = pp.Opts(
max_grad=32,
    grad_unit="mT/m",
    max_slew=130,
    slew_unit="T/m/s",
    rf_ringdown_time=100e-6,
    rf_dead_time=100e-6,
    adc_dead_time=10e-6,
)

seq = pp.Sequence(system)              # Create a new sequence object
                                       # Define FOV and resolution
dtp = np.array([2, 7, 5])
#dtp=2*dtp
#print(dtp)                           # to stay compatible with example 1
tp=np.cumsum(dtp)

flip = np.array([30, 40, 50])
rfphase = np.array([90, 0, 180])
slice_thickness = 50e-3            # slice
gr_area=250                               # repetition time TR
TP = 1e-3                        # time unit (=pulse duration)
dGt = 100e-6
# Create alpha-degree slice selection pulse and gradient
rf, gz, _ = pp.make_sinc_pulse(
  flip_angle = np.pi / 180,
  phase_offset = np.pi / 180,
  duration = TP-2*dGt,
  delay = dGt,
  slice_thickness = slice_thickness,
  apodization = 0.42,
  time_bw_product = 4,
  system = system,
  return_gz=True,
)
rf_ref=rf.signal
gz.rise_time = dGt
gz.fall_time = dGt
gz.delay = 0

Nx = round(gz.flat_time / system.grad_raster_time)
adc = pp.make_adc(
  num_samples = Nx,
  duration = gz.flat_time,
  delay=dGt,
  system=system,
)

# read gradient
gr = pp.make_trapezoid(
  channel="x",
  area=gr_area,
  duration=TP,
  system=system,
)

# y-spoiler gradient
gy = pp.make_trapezoid(
  channel="y",
  area=gr_area,
  duration=TP/2,
  system=system,
)

# z-spoiler gradient
gz = pp.make_trapezoid(
  channel="z",
  area=gr_area,
  duration=TP/2,
  system=system,
)


for nump in range (len(dtp)-1):
  rf.signal = rf_ref * flip[nump]
  rf.phase_offset = np.pi / 180 * rfphase[nump]
  seq.add_block(rf)
  gr = pp.make_trapezoid(
    channel="x",
    area=(dtp[nump])*gr_area,
    duration=(dtp[nump]-1)*TP,
    system=system,
  )
  seq.add_block(gr)


rf.signal = rf_ref * flip[nump]
rf.phase_offset = np.pi / 180 * rfphase[nump]
seq.add_block(rf)


gr = pp.make_trapezoid(
  channel="x",
  area=gr_area/2,
  duration=TP/2,
  system=system,
)
seq.add_block(gr)

gr = pp.make_trapezoid(
  channel="x",
  area=gr_area,
  duration=TP,
  system=system,
)
print(dtp[-1])
for nume in range (round(tp[-2])+1):
  seq.add_block(gr,adc)


seq.write(seq_filename)       # Write to pulseq file
seq.plot()

#[k_traj_adc, k_traj, t_excitation, t_refocusing, t_adc] = seq.calculate_kspace()

#seq.plot(time_range=[0,0.01])
seq.set_definition(key="FOV", value=Nx/gr_area)
#seq.install('siemens')


**Create MR-zero simulation object and run simulation**
loading and and definition of the simulation object has to be performed only once.

In [ ]:
# Download simulation object
!wget -O numerical_brain_cropped.mat https://github.com/mzaiss/MRTwin_pulseq/raw/mr0-core/data/numerical_brain_cropped.mat

In [ ]:
# Object definition
sz = [16, 16]
dB0 = 0
obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
obj_p = obj_p.interpolate(sz[0], sz[1], 1)
# Manipulate loaded data
obj_p.B0 += dB0
obj_p.D *= 0
obj_p.plot()
# Convert Phantom into simulation data
obj_p = obj_p.build()

In [ ]:
# MR zero simulation
seq0 = mr0.Sequence.from_seq_file(seq_filename)
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-5)
signal = mr0.execute_graph(graph, seq0, obj_p)
kdata = signal.reshape(1,-1,adc.num_samples) # Reshape to [N_coils, N_meas, N_adc]
print(len(signal))
plt.plot(abs(signal))